In [4]:
from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image


model_id = 'microsoft/Florence-2-base-ft'
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True).eval().cuda()
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)

def caption(task_prompt, text_input=None):
    if text_input is None:
        prompt = task_prompt
    else:
        prompt = task_prompt + text_input

    inputs = processor(text=prompt, images=image, return_tensors="pt").to("cuda:0")
    generated_ids = model.generate(
        input_ids=inputs["input_ids"].cuda(),
        pixel_values=inputs["pixel_values"].cuda(),
        max_new_tokens=1500,
        early_stopping=False,
        do_sample=False,
        num_beams=3,
    )
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
    parsed_answer = processor.post_process_generation(
        generated_text,
        task=task_prompt,
        image_size=(image.width, image.height)
    )

    return parsed_answer

config.json:   0%|          | 0.00/2.43k [00:00<?, ?B/s]

configuration_florence2.py:   0%|          | 0.00/15.1k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Florence-2-base-ft:
- configuration_florence2.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_florence2.py:   0%|          | 0.00/127k [00:00<?, ?B/s]

ImportError: This modeling file requires the following packages that were not found in your environment: flash_attn. Run `pip install flash_attn`

In [2]:
from transformers import AutoProcessor, AutoModelForCausalLM
import torch

# Set model ID and specify cache location
model_id = 'microsoft/Florence-2-base-ft'
local_dir = './models/Florence-2'

# Download model and save to local directory
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True, cache_dir=local_dir).to("cuda")
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True, cache_dir=local_dir)

print("Model and processor downloaded to:", local_dir)


ImportError: This modeling file requires the following packages that were not found in your environment: flash_attn. Run `pip install flash_attn`

In [5]:
import torch
print(torch.version.cuda)  # Kiểm tra phiên bản CUDA
print(torch.cuda.is_available())  # Kiểm tra xem CUDA có khả dụng không


11.8
True
